# Installations that cause frequent errors on TPU

In [1]:
!pip install seaborn
!pip install imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 5.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 5.9 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Set Environment

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
from tensorflow.keras.applications import ResNet50, InceptionResNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Average
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import gc

D0226 13:44:10.598210096      15 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0226 13:44:10.598233411      15 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0226 13:44:10.598237024      15 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0226 13:44:10.598239884      15 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0226 13:44:10.598242545      15 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0226 13:44:10.598245476      15 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0226 13:44:10.598248300      15 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0226 13:44:10.

TensorFlow version: 2.12.0


# Set Seed for Reproducibility

In [3]:
def set_seed(seed=21019):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Loading data splits and one-hot encoding

In [4]:
print('Loading data ...')
with open('/kaggle/input/revised-data-array/data_array.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

X = loaded_data['X']
y = loaded_data['y']

y_encoded = to_categorical(y)

print(f'Samples in Dataset: {len(y_encoded)}')

# Define class names
class_names = list(['Unidentifiable', 'Type 1', 'Type 2', 'Type 3'])

# Define the input shape
input_shape = X[0].shape

# Number of classes
num_classes = len(class_names)

# Class-wise sample count in training set
class_counts_train = np.bincount(np.argmax(y_encoded, axis=1))
for class_idx, count in enumerate(class_counts_train):
    print(f'Class {class_names[class_idx]}: {count} samples')

Loading data ...
Samples in Dataset: 8108
Class Unidentifiable: 2845 samples
Class Type 1: 1087 samples
Class Type 2: 2822 samples
Class Type 3: 1354 samples


# Detect TPU

In [5]:
# Detect and initialize the TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

# Creating Model Function

In [6]:
def create_model(input_shape, num_classes):
    # Define the input tensor
    inputs = Input(shape=input_shape)

    # Create the base ResNet model
    resnet_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)
    resnet_model.trainable = True

    # Create the base InceptionResNetV2 model
    inception_model = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=inputs)
    inception_model.trainable = True

    # Global average pooling and dense layer for ResNet
    resnet_x = GlobalAveragePooling2D()(resnet_model.output)
    resnet_x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.04))(resnet_x)

    # Global average pooling and dense layer for InceptionResNetV2
    inception_x = GlobalAveragePooling2D()(inception_model.output)
    inception_x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.04))(inception_x)

    # Ensemble layer by averaging the outputs
    ensemble = Average()([resnet_x, inception_x])

    # Output layer with softmax activation
    ensemble_output = Dense(num_classes, activation='softmax')(ensemble)

    # Create the model
    model = Model(inputs=inputs, outputs=ensemble_output)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Augmented Data Generator

In [7]:
# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=90,  # Rotate images randomly between -90 and 90 degrees
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=True  # Randomly flip images vertically
)

# Train Model

In [8]:
image_size = (256, 256)

# Number of folds (k)
k = 5

# Create a StratifiedKFold object with a random seed
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define a variable to keep track of the best model
best_model = None
best_accuracy = 0.0

# Initialize lists to store evaluation metrics for each fold
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Iterate over the folds
for fold, (train_idx, test_idx) in enumerate(kf.split(X, np.argmax(y_encoded, axis=1))):
    print(f'\n\033[1m{"="*30}\nFold {fold + 1}\n{"="*30}\033[0m')
    
    # Remove garbage to free up memory
    print(f'Freeing up memory!')
    gc.collect()
    
    x_train, x_test = X[train_idx], X[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Split the data into training and validation sets
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
    
    # Initialize SMOTE
    smote = SMOTE(random_state=2119)

    # Reshape the data for oversampling
    X_reshaped = x_train.reshape(-1, image_size[0] * image_size[1] * 3)

    # Apply oversampling
    X_resampled, y_resampled = smote.fit_resample(X_reshaped, np.argmax(y_train, axis=1))

    print("Shape of X_resampled:", X_resampled.shape)
    print("Shape of y_resampled:", y_resampled.shape)

    # Reshape the data back to the original shape
    X_resampled = X_resampled.reshape(-1, image_size[0], image_size[1], 3)

    x_train = X_resampled
    y_train = to_categorical(y_resampled, num_classes=num_classes)
    
    del X_resampled
    del y_resampled

    # Calculate class distribution after oversampling
    resampled_class_distribution = Counter(np.argmax(y_train, axis=1).tolist())
    print("Class distribution after oversampling:", resampled_class_distribution)


    print(f'Total Number of Samples after Oversampling: {len(y_train)+len(y_val)+len(y_test)}')
    print(f'Samples in Train Set: {len(y_train)}')
    print(f'Samples in Validation Set: {len(y_val)}')
    print(f'Samples in Test Set: {len(y_test)}')


    # Create the model within the TPU strategy scope
    with strategy.scope():
        model = create_model(input_shape, num_classes)
        
    # Create the Learning Rate callback
    lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-10)
    
    # Create a ModelCheckpoint callback to save the best model based on validation accuracy
    checkpoint_callback = ModelCheckpoint(f'best_model_fold_{fold + 1}.h5', 
                                          monitor='val_accuracy',
                                          mode='max',
                                          save_best_only=True,
                                          save_weights_only=True)
    
    # Define the early stopping callback to stop training if validation loss does not improve
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)
    
    
    # Define the proportion of data to augment
    augmentation_proportion = 0.25  # For example, augment 50% of the data

    # Determine the number of samples to augment
    num_samples_to_augment = int(len(x_train) * augmentation_proportion)

    # Randomly select samples to augment
    indices_to_augment = np.random.choice(len(x_train), num_samples_to_augment, replace=False)

    # Augment the selected samples
    x_augmented = []
    y_augmented = []
    for idx in indices_to_augment:
        image = x_train[idx]
        label = y_train[idx]
        # Apply augmentation
        augmented_images = datagen.flow(np.expand_dims(image, axis=0), np.expand_dims(label, axis=0), batch_size=1)
        for _ in range(2):  # Augment each sample twice
            augmented_image, augmented_label = next(augmented_images)
            x_augmented.append(augmented_image.squeeze())
            y_augmented.append(augmented_label.squeeze())

    # Convert lists to arrays
    x_augmented = np.array(x_augmented)
    y_augmented = np.array(y_augmented)

    # Concatenate augmented data with original data
    x_train_augmented = np.concatenate([x_train, x_augmented])
    y_train_augmented = np.concatenate([y_train, y_augmented])


    # Train the model with history tapping (keeps data in a dictionary)
    epochs = 40
    batch_size = 128
    history = model.fit(x_train_augmented, y_train_augmented,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[lr_callback, checkpoint_callback, early_stopping_callback],
                        shuffle=True)

    # Evaluate the model on the test set for this fold
    results = model.evaluate(x_test, y_test)
    print(f'\033[1mFold {fold + 1} - Test Accuracy = {results[1]:.4f}\033[0m')
    accuracy_scores.append(results[1])

    # Check if the current model is the best so far
    if results[1] > best_accuracy:
        best_accuracy = results[1]
        best_model = model

    # Predict on the test set and calculate precision, recall, and f1 score
    y_pred = model.predict(x_test)
    y_pred_class = np.argmax(y_pred, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(y_test_class, y_pred_class, average='macro')

    print(f'\033[1mFold {fold + 1} - Test Precision = {precision:.4f}\033[0m')
    print(f'\033[1mFold {fold + 1} - Test Recall = {recall:.4f}\033[0m')
    print(f'\033[1mFold {fold + 1} - Test F1 Score = {f1:.4f}\033[0m')
    
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Save the best model after all folds are complete
best_model.save('best_model.h5')

# Calculate mean and standard deviation for each metric
mean_accuracy = np.mean(accuracy_scores)
sd_accuracy = np.std(accuracy_scores)

mean_precision = np.mean(precision_scores)
sd_precision = np.std(precision_scores)

mean_recall = np.mean(recall_scores)
sd_recall = np.std(recall_scores)

mean_f1 = np.mean(f1_scores)
sd_f1 = np.std(f1_scores)

# Print mean ± sd form of metrics
print(f"\n\033[1mMean ± SD of Accuracy:\033[0m {mean_accuracy:.4f} ± {sd_accuracy:.4f}")
print(f"\n\033[1mMean ± SD of Precision:\033[0m {mean_precision:.4f} ± {sd_precision:.4f}")
print(f"\n\033[1mMean ± SD of Recall:\033[0m {mean_recall:.4f} ± {sd_recall:.4f}")
print(f"\n\033[1mMean ± SD of F1-Score:\033[0m {mean_f1:.4f} ± {sd_f1:.4f}")


Fold 1
Freeing up memory!
Shape of X_resampled: (7340, 196608)
Shape of y_resampled: (7340,)
Class distribution after oversampling: Counter({3: 1835, 0: 1835, 2: 1835, 1: 1835})
Total Number of Samples after Oversampling: 10260
Samples in Train Set: 7340
Samples in Validation Set: 1298
Samples in Test Set: 1622
219055592/219055592 [==============================] - 1s 0us/step
Epoch 1/40


2024-02-26 13:48:20.129650: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-02-26 13:48:23.257779: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


87/87 [==============================] - ETA: 0s - loss: 29.7635 - accuracy: 0.6621

2024-02-26 13:53:14.476833: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-02-26 13:53:15.127803: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


87/87 [==============================] - 420s 2s/step - loss: 29.7635 - accuracy: 0.6621 - val_loss: nan - val_accuracy: 0.3398 - lr: 1.0000e-04
Epoch 2/40
87/87 [==============================] - 24s 270ms/step - loss: nan - accuracy: 0.2490 - val_loss: nan - val_accuracy: 0.3398 - lr: 1.0000e-04
Epoch 3/40
87/87 [==============================] - 23s 270ms/step - loss: nan - accuracy: 0.2490 - val_loss: nan - val_accuracy: 0.3398 - lr: 1.0000e-05
Epoch 4/40
87/87 [==============================] - 23s 269ms/step - loss: nan - accuracy: 0.2490 - val_loss: nan - val_accuracy: 0.3398 - lr: 1.0000e-05
Epoch 5/40
51/51 [==============================] - 20s 52ms/step - loss: nan - accuracy: 0.3508
Fold 1 - Test Accuracy = 0.3508


...

87/87 [==============================] - ETA: 0s - loss: 29.7843 - accuracy: 0.6634

2024-02-26 14:56:13.347722: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-02-26 14:56:14.026947: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


87/87 [==============================] - 418s 2s/step - loss: 29.7843 - accuracy: 0.6634 - val_loss: nan - val_accuracy: 0.3382 - lr: 1.0000e-04
Epoch 2/40
87/87 [==============================] - 24s 276ms/step - loss: nan - accuracy: 0.2474 - val_loss: nan - val_accuracy: 0.3382 - lr: 1.0000e-04
Epoch 3/40
87/87 [==============================] - 24s 281ms/step - loss: nan - accuracy: 0.2474 - val_loss: nan - val_accuracy: 0.3382 - lr: 1.0000e-05
Epoch 4/40
87/87 [==============================] - 24s 276ms/step - loss: nan - accuracy: 0.2474 - val_loss: nan - val_accuracy: 0.3382 - lr: 1.0000e-05
Epoch 5/40
51/51 [==============================] - 20s 58ms/step - loss: nan - accuracy: 0.3510
Fold 5 - Test Accuracy = 0.3510


2024-02-26 14:59:04.392469: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-02-26 14:59:04.992878: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


51/51 [==============================] - 47s 455ms/step
Fold 5 - Test Precision = 0.0878
Fold 5 - Test Recall = 0.2500
Fold 5 - Test F1 Score = 0.1299

Mean ± SD of Accuracy: 0.5280 ± 0.2171

Mean ± SD of Precision: 0.3675 ± 0.3429

Mean ± SD of Recall: 0.4537 ± 0.2501

Mean ± SD of F1-Score: 0.3863 ± 0.3144


# Remove Redundant Files

In [9]:
# Specify the directory where the files are located
directory = '/kaggle/working/'

# Remove best model files from folds 1 to k
for fold in range(1, k+1):
    file_path = os.path.join(directory, f'best_model_fold_{fold}.h5')
    if os.path.exists(file_path):
        os.remove(file_path)